<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

In [4]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [5]:
pip install catboost

     |████████████████████████████████| 66.1MB 42kB/s 


In [6]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [7]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [8]:
# features

In [9]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

## Feature engineering

Feature Interaction

In [15]:
train['form_field1*formfield2'] = train['form_field1'] * train['form_field2']
train['form_field1/formfield2'] = train['form_field1'] / train['form_field2']

train['form_field1*formfield3'] = train['form_field1'] * train.form_field3
train['form_field1*formfield4'] = train.form_field1 * train.form_field4
train['form_field1*formfield5'] = train.form_field1 * train.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

train['form_field6+field7'] =  (train.form_field6 + train.form_field8)/2

train['form_field1*formfield6'] = train['form_field1'] * train['form_field6']
train['form_field2*formfield6'] = train['form_field2'] * train['form_field6']

train['form_field1*formfield9'] = train['form_field1'] * train['form_field9']
train['form_field2*formfield9'] = train['form_field2'] * train['form_field9']

train['form_field1*formfield37'] = train['form_field1'] * train['form_field37']
train['form_field2*formfield37'] = train['form_field2'] * train['form_field37']

train['form_field9*formfield6'] = train['form_field9'] * train['form_field6']
train['form_field37*formfield6'] = train['form_field37'] * train['form_field6']

train['form_field9*formfield37'] = train['form_field9'] * train['form_field37']
train['form_field1*formfield8'] = train['form_field1'] * train['form_field8']

In [16]:
test['form_field1*formfield2'] = test['form_field1'] * test['form_field2']
test['form_field1/formfield2'] = test['form_field1'] / test['form_field2']

test['form_field1*formfield3'] = test['form_field1'] * test.form_field3
test['form_field1*formfield4'] = test.form_field1 * test.form_field4
test['form_field1*formfield5'] = test.form_field1 * test.form_field5

test['form_field6+field7'] =  (test.form_field6 + test.form_field8)/2

test['form_field1*formfield6'] = test['form_field1'] * test['form_field6']
test['form_field2*formfield6'] = test['form_field2'] * test['form_field6']

test['form_field1*formfield9'] = test['form_field1'] * test['form_field9']
test['form_field2*formfield9'] = test['form_field2'] * test['form_field9']

test['form_field1*formfield37'] = test['form_field1'] * test['form_field37']
test['form_field2*formfield37'] = test['form_field2'] * test['form_field37']

test['form_field9*formfield6'] = test['form_field9'] * test['form_field6']
test['form_field37*formfield6'] = test['form_field37'] *test['form_field6']

test['form_field9*formfield37'] = test['form_field9'] * test['form_field37']
test['form_field1*formfield8'] = test['form_field1'] * test['form_field8']

In [17]:
X = train[features]
y = train['default_status']

In [18]:
from sklearn.model_selection import StratifiedKFold

In [19]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [20]:
# Specify number of folds
n_skf = 5
kf = StratifiedKFold(n_skf)
 
 
seed = 2020
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {
    'n_estimators': 5000,
    'learning_rate': 0.008,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [21]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU')
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.7882729	test: 0.7847513	best: 0.7847513 (0)	total: 107ms	remaining: 8m 55s
100:	learn: 0.8276219	test: 0.8236265	best: 0.8236671 (99)	total: 5.32s	remaining: 4m 17s
200:	learn: 0.8329277	test: 0.8278610	best: 0.8278610 (200)	total: 10.4s	remaining: 4m 7s
300:	learn: 0.8360772	test: 0.8302315	best: 0.8302315 (300)	total: 15.4s	remaining: 4m
400:	learn: 0.8384167	test: 0.8317417	best: 0.8317417 (400)	total: 20.3s	remaining: 3m 53s
500:	learn: 0.8402665	test: 0.8328101	best: 0.8328101 (500)	total: 25.3s	remaining: 3m 47s
600:	learn: 0.8416766	test: 0.8335868	best: 0.8335915 (599)	total: 30.3s	remaining: 3m 41s
700:	learn: 0.8430179	test: 0.8342006	best: 0.8342093 (699)	total: 35.3s	remaining: 3m 36s
800:	learn: 0.8442052	test: 0.8347365	best: 0.8347365 (800)	total: 40.2s	remaining: 3m 30s
900:	learn: 0.8451248	test: 0.8350822	best: 0.8350828 (895)	total: 45.1s	remaining: 3m 25s
1000:	learn: 0.8460628	test: 0.8354752	best: 0.8354752 (1000)	total: 50s	remaining: 3m 19s
1100:	lea

In [22]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'5 fold CV, score: 0.8402615431258239'

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [23]:
oof_prediction = pd.DataFrame(test_oofs).T

In [24]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [25]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5
0,0.342935,0.333955,0.365686,0.336708,0.330304
1,0.417118,0.361576,0.407666,0.322951,0.375386
2,0.340778,0.391729,0.427155,0.403827,0.346953
3,0.720198,0.734151,0.736173,0.751253,0.725948
4,0.200629,0.189707,0.175935,0.145036,0.166993


In [26]:
sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [28]:
from google.colab import files


In [29]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [31]:
sub.to_csv('submit3.csv', index = False)

In [32]:
files.download('submit3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.